In [1]:
import pandas as pd
import csv
import sqlite3
from datetime import datetime, timedelta

In [2]:
#Read in raw data
df1 = pd.read_excel('/Users/david.dibara/Downloads/2. Safety Incidents_Full Database.xlsx',skiprows = 5)
df2 = pd.read_csv('/Users/david.dibara/Downloads/SNES Raw Data.csv')

In [3]:
#Drop unecessary columns
df1 = df1.drop(['Member First Name','Member Last Name','Member Number', 'Member Email', 'Member Address', 'Member Phone', 'Member Location','Purchased From', 'Delivery By', 'Did an Injury Occur?', 'Describe the Injury', 'Medical Attention or Hospitalization?', 'Is Product under recall?', 
                'Recall Related?', 'Recall Repair Type', 'Was Fire or Smoke Reported?', 'Damage from Fire?', 'Property Damage?', 'Property Damage Details', 'Product in Use?', 'Details of Usage', 'Incident Description','Is Product being Returned?','Safety Team Assessment', 'Injury Type',
                'Investigation Status', 'Legal Disposition', 'Product Disposition', 'Closure Date', 'Closure Comments','Zendesk Ticket Number', 'Model'],axis=1)
df2 = df2.drop('Unnamed: 0',axis=1)

In [4]:
#Reformat date fields to same data type
df1['Date Incident Reported to Peloton'] = df1['Date Incident Reported to Peloton'].dt.date
df1['Product Delivery Date'] = df1['Product Delivery Date'].dt.date
df2['Originating Order Delivery Date'] = pd.to_datetime(df2['Originating Order Delivery Date'], format='%Y-%m-%d').dt.date
df2['Event Date Date'] = pd.to_datetime(df2['Event Date Date'], format='%Y-%m-%d').dt.date

In [5]:
#Rename all safety fields to remove spaces
df1 = df1.rename(columns={'Incident ID':'Incident_ID','Incident Date':'Incident_Date','Product Delivery Date':'Product_Delivery_Date','Date Incident Reported to Peloton':'Date_Incident_Reported_to_Peloton','Date Added to Safety Database':'Date_Added_to_Safety_Database','Serial Number(s)':'Serial_Numbers','Reported Issue':'Reported_Issue','Retrieve Product':'Retrieve_Product','Product System':'Product_System','Product Subsystem':'Product_Subsystem','Product Component':'Product_Component','Failure Mode':'Failure_Mode','Tag 1':'Tag_1','Tag 2':'Tag_2','Likelihood of Product being Origin':'Likelihood_of_Product_being_Origin','Property Damage Extent':'Property_Damage_Extent','Injury Extent':'Injury_Extent','Contact Week':'Contact_Week','Manufacturing Site':'Manufacturing_Site','Manufacturing Month':'Manufacturing_Month','Manufacturing Year':'Manufacturing_Year','Contact Month':'Contact_Month','Contact Year':'Contact_Year','Contact Quarter':'Contact_Quarter','Time From PL contact to form entry':'Time_From_PL_contact_to_form_entry','Age at Fault':'Age_at_Fault'})
df2 = df2.rename(columns={'Originating Order ID All Channels':'Originating_Order_ID_All_Channels','Originating Order Internal Product Type':'Originating_Order_Internal_Product_Type','Originating Order Delivery Date':'Originating_Order_Delivery_Date','Days Since Install':'Days_Since_Install','Event Date Date':'Event_Date_Date','Order ID':'Order_ID','Support Order Internal Product Type':'Support_Order_Internal_Product_Type','Support Order Priority Issue':'Support_Order_Priority_Issue','Support Order SKU List':'Support_Order_SKU_List','Serial Number Frame Chassis':'Serial_Number_Frame_Chassis','Is Claim Order (Yes / No)':'Is_Claim_Order','Is New/Refurbished':'Is_New_Refurbished','Subscription ID':'Subscription_ID'})

In [6]:
#Connect to database & create a cursor
conn = sqlite3.connect('quality_safety_join.db')
c = conn.cursor()

In [7]:
# Write the DataFrames to SQLite database tables
df1.to_sql('safety', conn, if_exists='replace', index=False)
df2.to_sql('quality', conn, if_exists='replace', index=False)

973180

In [8]:
#Add a column to safety data to indicate safety issue
c.execute('''ALTER TABLE safety
             ADD COLUMN Is_Safety_Issue BOOLEAN''')
c.execute('''UPDATE safety 
             SET Is_Safety_Issue = CASE WHEN Incident_ID IS NOT NULL THEN "Yes" ELSE "No" END''')

Data Joining

In [10]:
#Save column names for safety table in a list to use in formulas
column_names = c.execute("PRAGMA table_info(safety)")
safety_columns = []
for name in column_names:
    safety_columns.append(name[1])

In [11]:
#Save column names for quality table in a list to use in formulas
column_names = c.execute("PRAGMA table_info(quality)")
quality_columns = []
for name in column_names:
    quality_columns.append(name[1])

In [12]:
#Combine column names and null placeholders for use in union formula
safety_names = ', '.join(safety_columns)
quality_null = ', NULL AS '.join(quality_columns)
safety_str = ", NULL AS ".join([safety_names,quality_null])
quality_names = ', '.join(quality_columns)
safety_null = 'NULL AS ' + ', NULL AS '.join(safety_columns)
quality_str = ', ' + ', '.join([safety_null,quality_names])
trimmed_q_string = quality_str[2:]

In [13]:
#Union the two scaffolded tables into a single joined table
c.execute('CREATE TABLE safety_quality AS SELECT {} FROM safety UNION SELECT {} FROM quality'.format(safety_str,trimmed_q_string))

Post-Join Processing

In [15]:
#Created a unified product field and map internal product names to external product names
c.execute('ALTER TABLE safety_quality ADD COLUMN unified_product TEXT')
c.execute('''UPDATE safety_quality
           SET unified_product =
               CASE 
                   WHEN Incident_ID IS NOT NULL THEN Product_System
                   WHEN Originating_Order_Internal_Product_Type = "v1" THEN "Bike"
                   WHEN Originating_Order_Internal_Product_Type = "titan" THEN "Bike_Plus"
                   WHEN Originating_Order_Internal_Product_Type = "prism" THEN "Tread"
                   WHEN Originating_Order_Internal_Product_Type = "aurora" THEN "Tread_Plus"
                   WHEN Originating_Order_Internal_Product_Type = "caesar" THEN "Row"
                   WHEN Originating_Order_Internal_Product_Type = "hera" THEN "HRM"
                   WHEN Originating_Order_Internal_Product_Type = "tiger" THEN "Guide"
                   ELSE NULL
               END''')

In [16]:
#Create a merged delivery date field
c.execute('ALTER TABLE safety_quality ADD COLUMN merged_delivery_date DATE')
c.execute('''UPDATE safety_quality
                 SET merged_delivery_date = 
                     CASE
                         WHEN Incident_ID IS NOT NULL THEN Product_Delivery_Date
                         ELSE Originating_Order_Delivery_Date
                     END''')

In [17]:
#Create a merged event date field
c.execute('ALTER TABLE safety_quality ADD COLUMN merged_event_date DATE')
c.execute('''UPDATE safety_quality
                 SET merged_event_date = 
                     CASE
                         WHEN Incident_ID IS NOT NULL THEN Date_Incident_Reported_to_Peloton
                         ELSE Event_Date_Date
                     END''')

In [18]:
#Create new columns for delivery month and event month, keeping date formatting. Set month fields equal to the first date within each month
c.execute('ALTER TABLE safety_quality ADD COLUMN delivery_month DATE')
c.execute('ALTER TABLE safety_quality ADD COLUMN event_month DATE')
c.execute('''UPDATE safety_quality SET delivery_month = strftime('%Y-%m-01', merged_delivery_date)''')
c.execute('''UPDATE safety_quality SET event_month = strftime('%Y-%m-01', merged_event_date)''')

In [19]:
#Create an table for summary metrics by delivery date
c.execute('''DROP TABLE IF EXISTS delivery_summary_table''')
c.execute('''CREATE TABLE delivery_summary_table AS
                 SELECT delivery_month,
                        unified_product,
                        COUNT (DISTINCT CASE WHEN Originating_Order_Internal_Product_Type IS NOT NULL THEN Originating_Order_ID_All_Channels ELSE NULL END) AS quality_install_count,
                        COUNT (DISTINCT CASE WHEN Is_Claim_Order = "Yes" THEN Order_ID ELSE NULL END) AS quality_claim_count, 
                        COUNT (DISTINCT CASE WHEN Is_Safety_Issue = "Yes" THEN Incident_ID ELSE NULL END) AS safety_claim_count
                 FROM safety_quality
                 GROUP BY 1, 2
                 ORDER BY 1 DESC''')

In [20]:
#Create an table for summary metrics by event date
c.execute('''DROP TABLE IF EXISTS event_summary_table''')
c.execute('''CREATE TABLE event_summary_table AS
                 SELECT event_month,
                        unified_product,
                        COUNT (DISTINCT CASE WHEN Originating_Order_Internal_Product_Type IS NOT NULL THEN Originating_Order_ID_All_Channels ELSE NULL END) AS quality_install_count,
                        COUNT (DISTINCT CASE WHEN Is_Claim_Order = "Yes" THEN Order_ID ELSE NULL END) AS quality_claim_count, 
                        COUNT (DISTINCT CASE WHEN Is_Safety_Issue = "Yes" THEN Incident_ID ELSE NULL END) AS safety_claim_count
                 FROM safety_quality
                 GROUP BY 1, 2
                 ORDER BY 1 DESC''')

In [21]:
#Write the final tables to a dataframe
df_delivery = pd.read_sql_query('SELECT * FROM delivery_summary_table',conn)
df_event = pd.read_sql_query('SELECT * FROM event_summary_table', conn)

In [45]:
#Export results to .csv for Looker studio ingestion
df_delivery.to_csv('/Users/david.dibara/Downloads/delivery_table.csv',index=False)
df_event.to_csv('/Users/david.dibara/Downloads/event_table.csv',index=False)

In [23]:
#close the cursor and connection to the database
c.close()
conn.close()